In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql import DataFrame

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Function to load data from Unity Catalog tables
def load_data_from_catalog(table_name: str) -> DataFrame:
    try:
        df = spark.table(table_name)
        logger.info(f"Loaded data from {table_name}")
        return df
    except Exception as e:
        logger.error(f"Error loading data from {table_name}: {str(e)}")
        raise

# Function to load data from external MySQL database
def load_data_from_mysql(jdbc_url: str, query: str, user: str, password: str) -> DataFrame:
    try:
        df = spark.read.format("jdbc").option("url", jdbc_url).option("query", query).option("user", user).option("password", password).load()
        logger.info(f"Loaded data from MySQL with query: {query}")
        return df
    except Exception as e:
        logger.error(f"Error loading data from MySQL: {str(e)}")
        raise

# Load data from Unity Catalog tables
source_df_99 = load_data_from_catalog("genai_demo.jnj.pjotr_dev")
source_df_330 = load_data_from_catalog("genai_demo.jnj.textinput")
source_df_499 = load_data_from_catalog("genai_demo.jnj.c04_bseg")
source_df_500 = load_data_from_catalog("genai_demo.jnj.c04_ekpo")
source_df_384 = load_data_from_catalog("genai_demo.jnj.pjotr_in_pes")
source_df_291 = load_data_from_catalog("genai_demo.jnj.c19_ivl_data")
source_df_401 = load_data_from_catalog("genai_demo.jnj.pjotr_")
source_df_249 = load_data_from_catalog("genai_demo.jnj.pes_prep")

# Load data from external MySQL database
jdbc_url = "jdbc:mysql://your_mysql_server:3306/your_database"
mysql_user = dbutils.secrets.get(scope="your_scope", key="mysql_user")
mysql_password = dbutils.secrets.get(scope="your_scope", key="mysql_password")
source_df_403 = load_data_from_mysql(jdbc_url, "SELECT * FROM mysql_editable", mysql_user, mysql_password)
source_df_98 = load_data_from_mysql(jdbc_url, "SELECT * FROM mysql_editable", mysql_user, mysql_password)

# Apply transformations
try:
    # Example transformation: Conditional transformation on PJOTR ID
    transformed_df_484 = source_df_99.withColumn("PJOTR_ID", F.when(F.col("PJOTR_ID").isNull(), F.lit("Unknown")).otherwise(F.col("PJOTR_ID")))
    logger.info("Applied conditional transformation on PJOTR ID")

    # Example transformation: Filter on map field
    transformed_df_325 = source_df_330.filter(F.col("map_field") == "desired_value")
    logger.info("Filtered data on map field")

    # Example transformation: Join on PJOTR
    transformed_df_386 = source_df_99.join(source_df_401, "PJOTR_ID", "inner")
    logger.info("Performed join on PJOTR")

    # Example transformation: Summarize on LE code
    summarized_df_321 = source_df_500.groupBy("LE_code").agg(F.sum("amount").alias("total_amount"))
    logger.info("Summarized data on LE code")

    # Example transformation: Union on PJOTR
    union_df_389 = transformed_df_386.unionByName(source_df_384)
    logger.info("Performed union on PJOTR")

    # Example transformation: Formula on Invoice amount (USD)
    transformed_df_381 = source_df_291.withColumn("Invoice_amount_USD", F.col("Invoice_amount") * F.lit(1.1))  # Example conversion rate
    logger.info("Applied formula on Invoice amount (USD)")

except Exception as e:
    logger.error(f"Error during transformations: {str(e)}")
    raise

# Write outputs to Unity Catalog tables
try:
    target_catalog = "genai_demo"
    target_schema = "jnj"
    
    # Ensure schema exists
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {target_catalog}.{target_schema}")
    logger.info(f"Schema {target_catalog}.{target_schema} ensured")

    # Write transformed data to Unity Catalog target table
    transformed_df_484.write.format("delta").mode("overwrite").saveAsTable(f"{target_catalog}.{target_schema}.c03_pjotr")
    logger.info("Written transformed data to Unity Catalog table c03_pjotr")

    transformed_df_325.write.format("delta").mode("overwrite").saveAsTable(f"{target_catalog}.{target_schema}.c03_pjotr_midway")
    logger.info("Written transformed data to Unity Catalog table c03_pjotr_midway")

    union_df_389.write.format("delta").mode("overwrite").saveAsTable(f"{target_catalog}.{target_schema}.c03_unmapped_to_pjotr")
    logger.info("Written union data to Unity Catalog table c03_unmapped_to_pjotr")

except Exception as e:
    logger.error(f"Error writing outputs: {str(e)}")
    raise
